In [ ]:
def iterate_by_time_chunks(ds, chunk_size):
    time_len = ds.sizes["time"]
    for i in range(0, time_len, chunk_size):
        yield ds.isel(time=slice(i, i + chunk_size))

In [ ]:
import xarray
from tqdm import tqdm
from src.utils.id_powerline_endpoints import load_counties
import geopandas as gpd
from shapely import Point
from tqdm import tqdm
# with open("../data/gencast/source-era5_date-2022-01-01_res-1.0_levels-37_steps-40.nc", "rb") as f:
#     example = xarray.load_dataset(f).compute()

df = xarray.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-6h-64x33.zarr")
# df = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr")
example = df.sel(time=slice("2000-01-02", "2022-12-31"))



NameError: name 'iterate_by_time_chunks' is not defined

In [ ]:
n_steps = 100  # based on your estimate for ~50GB
for chunk in tqdm(iterate_by_time_chunks(df.sel(time=slice("1979", "2022")), n_steps)):
    df_chunk = chunk.to_dataframe().dropna()
    # process, save, or analyze
    # print(df_chunk.shape)
    lon_lat_df = df_chunk.index.to_frame(index=False)[['lon', 'lat']].drop_duplicates()
    lon_lat_df['lon_fixed'] = lon_lat_df['lon'].apply(lambda x: x - 360 if x > 180 else x)
    lon_lat_df['geometry'] = lon_lat_df.apply(lambda row: Point(row['lon_fixed'], row['lat']), axis=1)
